# Looking at cross section data in an ENDF file
## Overview

Cross section data in an ENDF file is stored in what is referred to as MF3 (aka 'file 3'). The format of each section in MF3 is relatively simple: it consists of a single ENDF record that gives tabulated energy and cross section values along with the associated interpolation data (this record is known as a TAB1 record). In addition to these data, an MF3 section also gives us the following:
- the ZA value of the target nuclide
- AWR: the atomic weight ratio of the target nuclide with respect to the neutron mass
- QI: the reaction Q-value
- QM: the mass-difference Q-value associated to the reaction
- LR: the complex break-up flag, which indicates that additional particles not specified by the reaction will be emitted (an example here would be decay of Be8 into two alpha particles)

The following is an example of the raw content of an unparsed MF3 section (in this case the n,4n cross section data for Pu239):

In [ ]:
import ENDFtk

tape = ENDFtk.tree.Tape.from_file( 'resources/n-Pu239.endf' )
section = tape.material( 9437 ).file( 3 ).section( 37 )
xs = section.parse()

print( section.content )

ENDFtk provides us with an abstracted interface that allows us access to the data in the MF3 section without having to know where the data exactly is in the ENDF record. The energy and cross section values can be extracted with the following interface:

In [ ]:
number_points = xs.NP
energies = xs.energies
values = xs.cross_sections

The interpolation data for the cross section data can be retrieved with the following interface:

In [ ]:
number_ranges = xs.NR
boundaries = xs.boundaries
interpolants = xs.interpolants

The additional data (the Q-values, AWR, etc.) can be retrieved with the following interface:

In [ ]:
za = xs.ZA
awr = xs.atomic_weight_ratio
qm = xs.mass_difference_qvalue
qi = xs.reaction_qvalue
lr = xs.complex_breakup

awr = xs.AWR
qm = xs.QM
qi = xs.QI
lr = xs.LR

## Application 1: extracting and reformatting cross section data
This is a very simple example where we extract the cross section data from a file for a givn reaction and just reformat the data into a more human readable table. In this particular example, we are looking at n,4n in Pu239 at 0 K.

In [ ]:
import ENDFtk

tape = ENDFtk.tree.Tape.from_file( 'resources/n-Pu239.endf' )

# extract the n,4n cross section for Pu239
section = tape.material( 9437 ).file( 3 ).section( 37 ).parse()

# print out a cross section table
print( '{:^12} | {:^12}'.format( 'energy', 'xs' ) )
print( '{:12} | {:12}'.format( '-' * 12, '-' * 12 ) )
for energy, value in zip( section.energies, section.cross_sections ) :
    
    print( '{:12.5e} | {:12.5e}'.format( energy, value ) )

## Application 2: plotting cross section data
One of the most common things a user wants to do with cross section data is simply looking at the data. The following example illustrates how we can use plotting utilities in Python to plot cross section data from several ENDF files. This particular case plots the total cross section for 5 nuclides (O16, U238, Pu239, Pu240 and Pu242) at 0 K.

In [ ]:
# use interactive plotting
# ensure that ipympl is installed -> "conda install -c conda-forge ipympl" (for anaconda)
# restart jupyter if required
%matplotlib widget

In [ ]:
import ENDFtk
import matplotlib.pyplot as plot

mt = 1 # MT = 1 is the total cross section

# open the ENDF file for each nuclide, get the reaction cross section and parse the data
U238 = ENDFtk.tree.Tape.from_file( 'resources/n-U238.endf' ).material( 9237 ).file( 3 ).section( mt ).parse()
Pu239 = ENDFtk.tree.Tape.from_file( 'resources/n-Pu239.endf' ).material( 9437 ).file( 3 ).section( mt ).parse()
Pu240 = ENDFtk.tree.Tape.from_file( 'resources/n-Pu240.endf' ).material( 9440 ).file( 3 ).section( mt ).parse()
Pu242 = ENDFtk.tree.Tape.from_file( 'resources/n-Pu242.endf' ).material( 9446 ).file( 3 ).section( mt ).parse()
O16 = ENDFtk.tree.Tape.from_file( 'resources/n-O16.endf' ).material( 825 ).file( 3 ).section( mt ).parse()

# plot the data
plot.figure()
plot.plot( U238.energies, U238.cross_sections, label = '${}^{238}$U', color = 'red', linewidth = 1.0 )
plot.plot( Pu239.energies, Pu239.cross_sections, label = '${}^{239}$Pu', color = 'green', linewidth = 1.0 )
plot.plot( Pu240.energies, Pu240.cross_sections, label = '${}^{240}$Pu', color = 'blue', linewidth = 1.0 )
plot.plot( Pu242.energies, Pu242.cross_sections, label = '${}^{242}$Pu', color = 'orange', linewidth = 1.0 )
plot.plot( O16.energies, O16.cross_sections, label = '${}^{16}$O', color = 'olive', linewidth = 1.0 )
plot.xlabel( 'Energy (eV)' )
plot.ylabel( 'Cross Section (b)' )
plot.title( 'ENDF/B-VII.1 total cross section at 293.6 K' )
plot.xlim( 1e-5, 10e+6 )
plot.ylim( 1e-1, 1e+6 )
plot.xscale( 'log' )
plot.yscale( 'log' )
plot.legend()
plot.show()